In [1]:
from archappl.client import ArchiverDataClient
from phantasy import MachinePortal
from datetime import datetime
%matplotlib inline
%matplotlib notebook
import matplotlib.pyplot as plt
import time
plt.rcParams['figure.dpi'] = 120
plt.rcParams['figure.figsize'] = (8, 6)

In [2]:
mp = MachinePortal("FRIB", "LINAC")
bpms = mp.get_elements(type="BPM")
client = ArchiverDataClient()
client.url = "http://epicsarchiver0.ftc:17668"

[12:59:51.579] INFO: phantasy.library.operation.core: Load new machine: 'FRIB', segment: 'LINAC'
[12:59:51.580] INFO: phantasy.library.parser.config: Importing 'FRIB' from '/usr/lib/phantasy-machines/FRIB'
[12:59:51.583] INFO: phantasy.library.parser.config: Loading machine configuration file: /usr/lib/phantasy-machines/FRIB/phantasy.ini
[12:59:51.615] INFO: phantasy.library.operation.lattice: Loading segment: 'LINAC'
[12:59:51.692] INFO: phantasy.library.operation.lattice: UNICORN policy will be loaded from /usr/lib/phantasy-machines/unicorn/unicorn-data.xlsx.
[12:59:51.692] INFO: phantasy.library.operation.lattice: Loading PV data from CSV/SQLite: /usr/lib/phantasy-machines/FRIB/linac/all_channels.csv
[12:59:51.693] INFO: phantasy.library.pv.datasource: Set source to be CSV table: all_channels.csv.
[12:59:51.895] INFO: phantasy.library.operation.lattice: Model data directory: /user/zhangt/phantasy_data/20201113/model_data/data_125951
[12:59:51.914] INFO: phantasy.library.operation.la

[12:59:52.780] WARNING: phantasy.library.settings.common: Skip LS1_WB06:BPM_D1604 [BPM] for settings.
[12:59:52.784] WARNING: phantasy.library.settings.common: Skip LS1_WB06:HMR_D1606 [HMR] for settings.
[12:59:52.789] WARNING: phantasy.library.settings.common: Skip LS1_CB07:BPM_D1614 [BPM] for settings.
[12:59:52.806] WARNING: phantasy.library.settings.common: Skip LS1_CB07:ND_D1633 [ND] for settings.
[12:59:52.808] WARNING: phantasy.library.settings.common: Skip LS1_CB07:BPM_D1634 [BPM] for settings.
[12:59:52.824] WARNING: phantasy.library.settings.common: Skip LS1_CB07:BPM_D1654 [BPM] for settings.
[12:59:52.836] WARNING: phantasy.library.settings.common: Skip LS1_WB07:BPM_D1668 [BPM] for settings.
[12:59:52.839] WARNING: phantasy.library.settings.common: Skip LS1_WB07:HMR_D1670 [HMR] for settings.
[12:59:52.844] WARNING: phantasy.library.settings.common: Skip LS1_CB08:BPM_D1677 [BPM] for settings.
[12:59:52.859] WARNING: phantasy.library.settings.common: Skip LS1_CB08:BPM_D1697 [B

[12:59:53.638] WARNING: phantasy.library.settings.common: Skip LS2_WC03:BPM_D2821 [BPM] for settings.
[12:59:53.641] WARNING: phantasy.library.settings.common: Skip LS2_WC03:HMR_D2823 [HMR] for settings.
[12:59:53.649] WARNING: phantasy.library.settings.common: Skip LS2_CC04:ND_D2828 [ND] for settings.
[12:59:53.671] WARNING: phantasy.library.settings.common: Skip LS2_WC04:BPM_D2861 [BPM] for settings.
[12:59:53.697] WARNING: phantasy.library.settings.common: Skip LS2_WC05:BPM_D2901 [BPM] for settings.
[12:59:53.709] WARNING: phantasy.library.settings.common: Skip LS2_CC06:ND_D2916 [ND] for settings.
[12:59:53.724] WARNING: phantasy.library.settings.common: Skip LS2_WC06:BPM_D2941 [BPM] for settings.
[12:59:53.727] WARNING: phantasy.library.settings.common: Skip LS2_WC06:HMR_D2942 [HMR] for settings.
[12:59:53.752] WARNING: phantasy.library.settings.common: Skip LS2_WC07:BPM_D2981 [BPM] for settings.
[12:59:53.758] WARNING: phantasy.library.settings.common: Skip LS2_CC08:ND_D2987 [ND] 

In [3]:
def get_data(pv, ts_from, ts_to):
    data = client.get_data(pv,
                           ifrom=ts_from, #'2020-10-09T12:00:00.000000-05:00',
                           to=ts_to) #'2020-10-09T12:20:00.000000-05:00')
    if len(data.iloc[:, 0]) == 1:
        return None
    data.drop(columns=['severity', 'status'], inplace=True)
    data.rename(columns={'val': pv}, inplace=True)
    return data

# 185 MeV/u, Xe, 2020-10-09

## 20:05 - 21:35

In [4]:
ts_from = "2020-10-09T20:05:00.000000-04:00"
ts_to = "2020-10-09T21:35:00.000000-04:00"
field_list = ['X', 'Y', 'PHA']

In [5]:
t0 = time.time()
data = None
for i in bpms:
    for field in field_list:
        pv = i.pv(field=field)[0]
        data_ = get_data(pv, ts_from, ts_to)
        if data_ is None:
            continue
        if data is None:
            data = data_
        else:
            data = data.join(data_, how='outer')
data.fillna(method='ffill', inplace=True)
data.index.name = 'Timestamp'
print(f"Data fetching is done, cost {time.time() - t0} seconds.")

KeyboardInterrupt: 

In [6]:
data.memory_usage().sum() / 1024 / 1024 # MB

0.02032470703125

In [7]:
data.head(100)

,FE_MEBT:BPM_D1056:XPOS_RD,FE_MEBT:BPM_D1056:YPOS_RD,FE_MEBT:BPM_D1056:PHASE_RD,FE_MEBT:BPM_D1072:XPOS_RD,FE_MEBT:BPM_D1072:YPOS_RD,FE_MEBT:BPM_D1072:PHASE_RD,FE_MEBT:BPM_D1094:XPOS_RD,FE_MEBT:BPM_D1094:YPOS_RD,FE_MEBT:BPM_D1094:PHASE_RD,FE_MEBT:BPM_D1111:XPOS_RD,...,LS1_CA02:BPM_D1177:PHASE_RD,LS1_WA02:BPM_D1188:XPOS_RD,LS1_WA02:BPM_D1188:YPOS_RD,LS1_WA02:BPM_D1188:PHASE_RD,LS1_CA03:BPM_D1196:XPOS_RD,LS1_CA03:BPM_D1196:YPOS_RD,LS1_CA03:BPM_D1196:PHASE_RD,LS1_CA03:BPM_D1211:XPOS_RD,LS1_CA03:BPM_D1211:YPOS_RD,LS1_CA03:BPM_D1211:PHASE_RD
Timestamp,,,,,,,,,,,,,,,,,,,,,
2020-11-12 12:30:06.088350296-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-11-12 12:30:06.088374853-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-11-12 12:30:06.099469423-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-11-12 12:30:06.099479914-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-11-12 12:30:06.099929571-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-11-12 12:30:06.099934340-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-11-12 12:30:06.119085312-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-11-12 12:30:06.119110107-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-11-12 12:30:06.129732370-05:00,NaN,NaN,NaN,NaN,NaN,NaN,-0.3175,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
data_1 = data.resample('1S').ffill()

In [9]:
data_1.memory_usage().sum() / 1024 / 1024 # MB

13.392066955566406

In [10]:
data_1.dropna(inplace=True)

In [11]:
data_1

,FE_MEBT:BPM_D1056:XPOS_RD,FE_MEBT:BPM_D1056:YPOS_RD,FE_MEBT:BPM_D1056:PHASE_RD,FE_MEBT:BPM_D1072:XPOS_RD,FE_MEBT:BPM_D1072:YPOS_RD,FE_MEBT:BPM_D1072:PHASE_RD,FE_MEBT:BPM_D1094:XPOS_RD,FE_MEBT:BPM_D1094:YPOS_RD,FE_MEBT:BPM_D1094:PHASE_RD,FE_MEBT:BPM_D1111:XPOS_RD,...,LS2_WD12:BPM_D3924:PHASE_RD,FS2_BTS:BPM_D3943:XPOS_RD,FS2_BTS:BPM_D3943:YPOS_RD,FS2_BTS:BPM_D3943:PHASE_RD,FS2_BTS:BPM_D3958:XPOS_RD,FS2_BTS:BPM_D3958:YPOS_RD,FS2_BTS:BPM_D3958:PHASE_RD,FS2_BTS:BPM_D4006:XPOS_RD,FS2_BTS:BPM_D4006:YPOS_RD,FS2_BTS:BPM_D4006:PHASE_RD
Timestamp,,,,,,,,,,,,,,,,,,,,,
2020-10-09 20:05:00-04:00,0.404377,0.880039,77.569656,0.506108,-0.478523,-25.983869,-0.747265,-1.459981,-15.953848,-0.430815,...,-40.441614,2.821151,-1.790823,-60.777034,2.799540,-1.620521,-4.901362,2.937921,-0.117102,-24.264323
2020-10-09 20:05:01-04:00,0.501800,0.940125,77.443820,0.411774,-0.472942,-25.967486,-0.681081,-1.575400,-16.008769,-0.513283,...,-40.583194,2.419515,-1.739073,-60.375349,2.530807,-1.879712,-4.777756,3.852751,-0.110813,-23.995510
2020-10-09 20:05:02-04:00,0.505281,0.882828,77.371242,0.402470,-0.465830,-26.009886,-0.687470,-1.549598,-16.072344,-0.557424,...,-40.297675,2.573219,-2.545311,-61.203757,2.931046,-1.520655,-4.800355,2.944201,-0.942236,-25.068053
2020-10-09 20:05:03-04:00,0.512273,0.967212,77.330446,0.456395,-0.495318,-26.060279,-0.592182,-1.557194,-15.751991,-0.575431,...,-40.186573,2.611325,-1.877551,-60.677281,2.676378,-1.249955,-4.329805,2.483259,0.202339,-24.441554
2020-10-09 20:05:04-04:00,0.518947,0.953240,77.409251,0.309038,-0.512961,-26.023141,-0.524726,-1.547556,-15.682293,-0.469728,...,-40.274764,2.627293,-2.024369,-60.610234,2.589994,-1.539110,-4.642922,2.722641,0.403299,-24.419198
2020-10-09 20:05:05-04:00,0.519937,0.990873,77.274733,0.431507,-0.555529,-25.906895,-0.565463,-1.500532,-15.629282,-0.574737,...,-40.581978,2.821685,-1.567823,-61.102879,2.860524,-1.683764,-5.022572,2.853548,-0.336057,-23.950341
2020-10-09 20:05:06-04:00,0.567682,1.009830,77.278301,0.410476,-0.523964,-25.877977,-0.587557,-1.587923,-15.595935,-0.588239,...,-40.391986,2.294768,-2.138519,-60.687508,2.780890,-1.571500,-4.887581,2.052996,-1.100175,-24.105790
2020-10-09 20:05:07-04:00,0.544371,0.934782,77.328748,0.391318,-0.515691,-26.082514,-0.669504,-1.554449,-15.664730,-0.549653,...,-40.280726,2.480944,-1.621719,-61.191694,2.975289,-1.774666,-4.844286,3.171581,0.496668,-24.329516
2020-10-09 20:05:08-04:00,0.531805,0.940514,77.232677,0.393160,-0.475337,-25.798307,-0.621531,-1.579057,-15.759868,-0.525639,...,-39.940128,2.552457,-2.173073,-60.565122,2.416195,-1.585380,-5.001156,3.067551,-0.995646,-24.977896


In [12]:
data_1.shape

(5400, 324)

In [13]:
def save_dataset(data, time_from, time_to):
    filename = "185MeV_2020-10-09_{0}-{1}_Xe.csv".format(time_from.replace(':', ''), time_to.replace(':', ''))
    data[f'2020-10-09 {time_from}:00-04:00':f'2020-10-09 {time_to}:00-04:00'].to_csv(filename)

In [14]:
save_dataset(data_1, '20:05', '21:35')

In [9]:
pv1 = bpms[0].pv(field='X')[0]
pv2 = bpms[0].pv(field='Y')[0]

In [16]:
get_data(pv1, ts_from, ts_to)

,FE_MEBT:BPM_D1056:XPOS_RD
ts,
2020-10-09 20:04:59.949027061-04:00,0.404377
2020-10-09 20:05:00.948902130-04:00,0.501800
2020-10-09 20:05:01.949081898-04:00,0.505281
2020-10-09 20:05:02.948755264-04:00,0.512273
2020-10-09 20:05:03.949242115-04:00,0.518947
2020-10-09 20:05:04.949413776-04:00,0.519937
2020-10-09 20:05:05.948706627-04:00,0.567682
2020-10-09 20:05:06.948514462-04:00,0.544371
2020-10-09 20:05:07.949481964-04:00,0.531805


In [17]:
get_data(pv2, ts_from, ts_to)

,FE_MEBT:BPM_D1056:YPOS_RD
ts,
2020-10-09 20:04:59.949045658-04:00,0.880039
2020-10-09 20:05:00.948915005-04:00,0.940125
2020-10-09 20:05:01.949091434-04:00,0.882828
2020-10-09 20:05:02.948765993-04:00,0.967212
2020-10-09 20:05:03.949255466-04:00,0.953240
2020-10-09 20:05:04.949422121-04:00,0.990873
2020-10-09 20:05:05.948720455-04:00,1.009830
2020-10-09 20:05:06.948542118-04:00,0.934782
2020-10-09 20:05:07.949500799-04:00,0.940514


In [91]:
get_data('LS1_CB11:BPM_D1889:XPOS_RD', ts_from, ts_to)

,LS1_CB11:BPM_D1889:XPOS_RD
ts,
2020-10-09 12:14:59.568955898-04:00,-1.705230
2020-10-09 12:15:00.569184542-04:00,-1.734427
2020-10-09 12:15:01.569124937-04:00,-1.731108
2020-10-09 12:15:02.568714142-04:00,-1.752989
2020-10-09 12:15:03.568912745-04:00,-1.641288
2020-10-09 12:15:04.569242001-04:00,-1.662973
2020-10-09 12:15:05.569145679-04:00,-1.689221
2020-10-09 12:15:06.568968773-04:00,-1.739928
2020-10-09 12:15:07.570309639-04:00,-1.745748


In [10]:
ts = "2020-10-15T20:55:13.000000-04:00"
r = client.get_data_at_time([pv1, pv2], ts=ts)

In [11]:
r

{'FE_MEBT:BPM_D1056:YPOS_RD': {'severity': 0,
  'val': -0.12063862359699784,
  'nanos': 390683491,
  'secs': 1602809712,
  'status': 0},
 'FE_MEBT:BPM_D1056:XPOS_RD': {'severity': 0,
  'val': 0.6719326975404453,
  'nanos': 390668062,
  'secs': 1602809712,
  'status': 0}}

In [67]:
ts_epoch = 1602260099
datetime.fromtimestamp(ts_epoch).isoformat() + "-04:00"

'2020-10-09T12:14:59-04:00'